In [192]:
! pip install groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.1 MB/s eta 0:00:00


In [193]:
from google.colab import userdata
groq_token=userdata.get("groq")

In [194]:
from groq import Groq

In [195]:
client=Groq(
    api_key=groq_token
)

In [196]:
response=client.chat.completions.create(
    messages=[
        {
            "role":"user",
            "content":"Hello"
        }
    ],
    model="llama3-8b-8192",

)
response.choices[0].message.content

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [197]:
from typing import List
from pydantic import BaseModel
import json
class DataFormat(BaseModel):
  tokens:List[str]
  labels:List[str]

In [198]:
content="""
Your role is to generate datasets that can be used to train machine learning models to accurately identify and extract food names from text.
The datasets you create should follow these guidelines:

Tokens: Each dataset should contain a list of tokens representing the text. These tokens can be individual words, phrases, or other textual elements.
Labels: For each token in the dataset, there should be a corresponding label indicating whether the token represents a food name or not. The labels should use the standard BIO (beginning, inside, outside) tagging scheme, where:

"B-FOOD" indicates the token is the beginning of a food name.
"I-FOOD" indicates the token is inside a food name.
"O" indicates the token is not part of a food name.
JSON Format: The dataset should be provided in a JSON format, with the following structure: {
      "tokens": [],
      "labels": []
    }
Variety: The datasets should cover a wide range of food items, including common dishes, ingredients, and specialized or regional foods. The datasets should also include examples of food names that are part of longer phrases or sentences.
Realism: The text in the datasets should be representative of real-world usage, including variations in spelling, capitalization, and context. The datasets should not be artificially constructed or unrealistically simple.
Quality: The datasets should be thoroughly reviewed for accuracy and consistency. The food name annotations should be carefully checked to ensure they correctly identify all relevant food items.

When generating the datasets, you should draw from a diverse set of sources, such as recipe websites, restaurant menus, food blogs, and social media posts. You may also need to perform additional data cleaning, preprocessing, and annotation to ensure the datasets meet the required standards.
The response has only the json format dataset.
"""

In [235]:
from tqdm import tqdm
def generate_data(num_samples):
  data=[]
  for i in tqdm(range(num_samples)):
    try:
      json_data=client.chat.completions.create(
        messages=[
            {
                "role":"system",
                "content":content
              },
            {
                "role":"user",
                "content":"generate only 1 record in the dataset"
            },
            {
                "role":"assistant",
                "content":'''[
                  {"tokens": ["First", "time", "having", "vegan", "sushi", "yesterday", "since", "are", "in", "Manchester", "for", "a", "short", "time","."],"labels": ["O", "O", "O", "B-FOOD", "I-FOOD", "O", "O", "O", "O", "O", "O", "O", "O", "O","O"]},
                  {'tokens': ['I','love','to','cook','pasta','with','garlic','and','basil.'],'labels': ['O', 'O', 'O', 'O', 'B-FOOD', 'O', 'B-FOOD', 'O', 'B-FOOD']}
                  ]
                '''
            },
            {
                "role":"user",
                "content":"generate another one"
            }
          ],
        response_format={"type":"json_object"},
        temperature=0.4,
        top_p=1,
        model="llama3-groq-70b-8192-tool-use-preview"
    )
      data.append(json.loads(json_data.choices[0].message.content))
    except Exception as e:
      print(e)
      continue
  return data

In [238]:
gen_data=generate_data(200)

  8%|▊         | 16/200 [00:33<23:55,  7.80s/it]

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '```json\n{\n   "tokens": ["I", "tried", "the", "new", "vegan", "restaurant", "in", "town", "last", "night", "and", "it", "was", "amazing", "the", "tofu", "tacos", "were", "so", "delicious", "I", "will", "definitely", "be", "back", "for", "more", "of", "that", "yummy", "food", "and", "the", "service", "was", "top", "notch", "too", "the", "server", "was", "very", "friendly", "and", "helpful", "with", "the", "menu", "options", "as", "well", "as", "the", "wine", "list", "great", "experience", "overall", "I", "highly", "recommend", "this", "place", "to", "anyone", "looking", "for", "a", "great", "dining", "experience", "with", "a", "variety", "of", "options", "for", "all", "dietary", "needs", "and", "preferences", "definitely", "check", "it", "out", "you", "won\\\'t", "b

 10%|█         | 21/200 [01:00<26:44,  8.97s/it]

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '```json\n{\n   "tokens": ["The", "best", "tacos", "in", "the", "world", "are", "at", "El", "Taco", "Loco", "in", "LA", "and", "the", "best", "sushi", "is", "at", "Sushi", "Yoshi", "in", "NYC", "for", "me", "and", "my", "family", "we", "love", "to", "eat", "out", "and", "try", "new", "restaurants", "and", "cuisines", "all", "the", "time", "I", "have", "tried", "so", "many", "types", "of", "food", "and", "I", "love", "them", "all", "but", "I", "think", "my", "favorite", "is", "sushi", "and", "tacos", "I", "love", "sushi", "and", "tacos", "so", "much", "I", "can", "eat", "them", "every", "day", "and", "never", "get", "tired", "of", "them", "I", "am", "so", "happy", "to", "have", "such", "great", "restaurants", "to", "go", "to", "and", "enjoy", "my", "favorite", "foods"

 16%|█▌        | 32/200 [01:29<21:59,  7.85s/it]

...

 95%|█████████▌| 190/200 [12:39<02:09, 12.92s/it]

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '```json\n{\n   "tokens": ["I", "tried", "the", "new", "vegan", "pizza", "place", "last", "night", "and", "the", "vegan", "cheese", "was", "amazing", "definitely", "going", "back", "there", "soon", "for", "more", "vegan", "food", "options", "like", "vegan", "sushi", "and", "vegan", "burgers", "and", "vegan", "fries", "and", "vegan", "milkshakes", "and", "vegan", "ice", "cream", "and", "vegan", "cake", "and", "vegan", "cookies", "and", "vegan", "brownies", "and", "vegan", "muffins", "and", "vegan", "bagels", "and", "vegan", "donuts", "and", "vegan", "pancakes", "and", "vegan", "waffles", "and", "vegan", "smoothies", "and", "vegan", "salads", "and", "vegan", "soups", "and", "vegan", "stews", "and", "vegan", "curries", "and", "vegan", "stir", "fries", "and", "vegan", "d

 96%|█████████▌| 191/200 [13:03<02:27, 16.43s/it]

Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '```json\n{\n   "tokens": ["Tried", "the", "new", "vegan", "pizza", "at", "the", "local", "restaurant", "last", "night", "and", "it", "was", "amazing", "!", "The", "mushroom", "sauce", "on", "top", "was", "a", "game", "changer", "for", "me", "as", "a", "non-vegan", "usually", "I", "would", "not", "like", "mushroom", "but", "this", "was", "perfectly", "balanced", "with", "the", "cheese", "and", "the", "crust", "was", "so", "fluffy", "and", "fresh", "I", "highly", "recommend", "this", "dish", "to", "anyone", "who", "is", "a", "foodie", "or", "just", "looking", "to", "try", "something", "new", "and", "delicious", "!", "Will", "definitely", "be", "back", "for", "more", "of", "this", "dish", "and", "to", "try", "other", "options", "on", "the", "menu", "as", "well", "as", 

100%|██████████| 200/200 [13:08<00:00,  3.94s/it]


In [239]:
len(gen_data)

168

In [240]:
import pandas as pd
df=pd.DataFrame(gen_data)
df

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
1,"[I'm, thinking, of, trying, out, some, new, re...","[O, O, O, O, O, O, O, O, O, O, O, B-FOOD, I-FO..."
2,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
3,"[I, tried, the, new, vegan, mushroom, burger, ...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
...,...,...
163,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
164,"[I, tried, the, new, vegan, restaurant, last, ...","[O, O, O, O, B-FOOD, O, O, O, O, O, O, O, O, B..."
165,"[I, had, a, great, dinner, last, night, at, th...","[O, O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD,..."
166,"[I, tried, a, new, recipe, last, night, for, c...","[O, O, O, O, O, O, O, O, B-FOOD, I-FOOD, O, B-..."


In [242]:
df.shape

(168, 2)

In [241]:
df["tokens"].duplicated().sum()

35

In [243]:
df.drop_duplicates(subset="tokens",inplace=True)

In [244]:
df.shape

(133, 2)

In [245]:
df.isna().sum()

,0
tokens,0
labels,0


In [246]:
df.head()

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
1,"[I'm, thinking, of, trying, out, some, new, re...","[O, O, O, O, O, O, O, O, O, O, O, B-FOOD, I-FO..."
3,"[I, tried, the, new, vegan, mushroom, burger, ...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
5,"[The, best, Thai, restaurant, in, Manchester, ...","[O, O, B-FOOD, O, O, O, O, O, B-FOOD, I-FOOD, ..."


In [249]:
df=df[df["tokens"].apply(len)==df['labels'].apply(len)]
df.head()

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
7,"[The, best, pasta, in, Manchester, is, at, Lui...","[O, O, B-FOOD, O, O, O, O, B-FOOD, B-FOOD, B-F..."
8,"[I, tried, to, make, a, vegan, pizza, yesterda...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
12,"[I, tried, the, new, vegan, pasta, dish, at, t...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."


In [251]:
other_df=pd.read_csv("clean_data_ner.csv")
other_df.head()

,Unnamed: 0,tokens,labels,sentences
0,0,"['I', 'love', 'to', 'eat', 'sushi', 'at', 'the...","['O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', ...",I love to eat sushi at the Japanese restaurant.
1,2,"['The', 'best', 'way', 'to', 'enjoy', 'pasta',...","['O', 'O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', ...",The best way to enjoy pasta is with a good sauce.
2,4,"['I', 'love', 'to', 'eat', 'sushi', 'at', 'my'...","['O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', ...",I love to eat sushi at my favorite restaurant.
3,6,"['The', 'best', 'Italian', 'restaurant', 'in',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FO...",The best Italian restaurant in town serves del...
4,7,"['I', 'love', 'eating', 'sushi', 'at', 'my', '...","['O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', 'O']",I love eating sushi at my favorite restaurant.


In [253]:
df_other=other_df[["tokens","labels"]]
df_other

,tokens,labels
0,"['I', 'love', 'to', 'eat', 'sushi', 'at', 'the...","['O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', ..."
1,"['The', 'best', 'way', 'to', 'enjoy', 'pasta',...","['O', 'O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', ..."
2,"['I', 'love', 'to', 'eat', 'sushi', 'at', 'my'...","['O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', ..."
3,"['The', 'best', 'Italian', 'restaurant', 'in',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FO..."
4,"['I', 'love', 'eating', 'sushi', 'at', 'my', '...","['O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', 'O']"
...,...,...
361,"['Today', 'I', 'had', 'a', 'delicious', 'chick...","['O', 'O', 'O', 'O', 'O', 'B-FOOD', 'I-FOOD', ..."
362,"['The', 'best', 'Italian', 'restaurant', 'in',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FO..."
363,"['The', 'best', 'thing', 'about', 'summer', 'i...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FO..."
364,"[""I'm thinking of making a"", 'Chicken Parmesan...","['O', 'B-FOOD', 'I-FOOD']"


In [255]:
new_df=pd.concat([df,df_other])
new_df.head()

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
7,"[The, best, pasta, in, Manchester, is, at, Lui...","[O, O, B-FOOD, O, O, O, O, B-FOOD, B-FOOD, B-F..."
8,"[I, tried, to, make, a, vegan, pizza, yesterda...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
12,"[I, tried, the, new, vegan, pasta, dish, at, t...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."


In [262]:
df_other[df_other["tokens"].apply(len)==df_other['labels'].apply(len)]

,tokens,labels


In [294]:
import ast
def turn_to_list(x):
  try:
    out=ast.literal_eval(x)
    return out
  except:
    return x
new_df["labels"]=new_df["labels"].apply(turn_to_list)
new_df["tokens"]=new_df["tokens"].apply(turn_to_list)
new_df

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
7,"[The, best, pasta, in, Manchester, is, at, Lui...","[O, O, B-FOOD, O, O, O, O, B-FOOD, B-FOOD, B-F..."
8,"[I, tried, to, make, a, vegan, pizza, yesterda...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
12,"[I, tried, the, new, vegan, pasta, dish, at, t...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."
...,...,...
361,"[Today, I, had, a, delicious, chicken, curry, ...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
362,"[The, best, Italian, restaurant, in, town, ser...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD]"
363,"[The, best, thing, about, summer, is, the, fre...","[O, O, O, O, O, O, O, O, B-FOOD, I-FOOD, O, O,..."
364,"[I'm thinking of making a, Chicken Parmesan, f...","[O, B-FOOD, I-FOOD]"


In [295]:
new_df[new_df["tokens"].apply(len)==new_df['labels'].apply(len)]

,tokens,labels
0,"[I, love, to, cook, pasta, with, garlic, and, ...","[O, O, O, O, B-FOOD, O, B-FOOD, O, B-FOOD]"
4,"[I, am, thinking, of, cooking, a, delicious, T...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD, O,..."
7,"[The, best, pasta, in, Manchester, is, at, Lui...","[O, O, B-FOOD, O, O, O, O, B-FOOD, B-FOOD, B-F..."
8,"[I, tried, to, make, a, vegan, pizza, yesterda...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
12,"[I, tried, the, new, vegan, pasta, dish, at, t...","[O, O, O, O, B-FOOD, I-FOOD, I-FOOD, O, O, O, ..."
...,...,...
361,"[Today, I, had, a, delicious, chicken, curry, ...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,..."
362,"[The, best, Italian, restaurant, in, town, ser...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD]"
363,"[The, best, thing, about, summer, is, the, fre...","[O, O, O, O, O, O, O, O, B-FOOD, I-FOOD, O, O,..."
364,"[I'm thinking of making a, Chicken Parmesan, f...","[O, B-FOOD, I-FOOD]"


In [307]:
def filters(x):
  out=x[0].split(" ")
  return len(out)==1
new_df=new_df[new_df["tokens"].map(filters)]

In [308]:
from google.colab import sheets

sheet = sheets.InteractiveSheet(df=new_df)

https://docs.google.com/spreadsheets/d/1mmcgfQeeAjSZhstbqxsXDPF9gcI57DIkom7cH1NUA2Q#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


In [309]:
new_df.to_csv("data_food_NER.csv")

In [310]:
from google.colab import files
files.download("data_food_NER.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [176]:
import swifter
df["sentences"]=df["tokens"].swifter.apply(lambda x: " ".join(x))
df.head()

Pandas Apply:   0%|          | 0/807 [00:00<?, ?it/s]

,tokens,labels,sentences
0,"[I, love, to, eat, sushi, at, the, Japanese, r...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at the Japanese restaurant.
1,"[The, best, pizza, in, town, is, at, Luigi's, ...","[O, O, B-FOOD, O, O, O, O, B-FOOD, I-FOOD, O, ...",The best pizza in town is at Luigi's Trattoria...
2,"[The, best, way, to, enjoy, pasta, is, with, a...","[O, O, O, O, O, B-FOOD, O, O, O, O, O]",The best way to enjoy pasta is with a good sauce.
3,"[The, best, way, to, start, your, day, is, wit...","[O, O, O, O, O, O, O, O, O, O, O, B-FOOD, I-FO...",The best way to start your day is with a delic...
4,"[I, love, to, eat, sushi, at, my, favorite, re...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at my favorite restaurant.


In [187]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1tGlkaHbhPIrWs3l_v4Ii9vksd7qx2OnLiQZ3IGxUQbo#gid=0


In [179]:
df=df.drop_duplicates(subset="sentences")
df.head()

,tokens,labels,sentences
0,"[I, love, to, eat, sushi, at, the, Japanese, r...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at the Japanese restaurant.
1,"[The, best, pizza, in, town, is, at, Luigi's, ...","[O, O, B-FOOD, O, O, O, O, B-FOOD, I-FOOD, O, ...",The best pizza in town is at Luigi's Trattoria...
2,"[The, best, way, to, enjoy, pasta, is, with, a...","[O, O, O, O, O, B-FOOD, O, O, O, O, O]",The best way to enjoy pasta is with a good sauce.
3,"[The, best, way, to, start, your, day, is, wit...","[O, O, O, O, O, O, O, O, O, O, O, B-FOOD, I-FO...",The best way to start your day is with a delic...
4,"[I, love, to, eat, sushi, at, my, favorite, re...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at my favorite restaurant.


In [184]:
df.to_csv("data_food_ner.csv")

In [185]:
df.shape

(535, 3)

In [198]:
df=df[df["labels"].apply(len)==df["tokens"].apply(len)]
df

,tokens,labels,sentences
0,"[I, love, to, eat, sushi, at, the, Japanese, r...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at the Japanese restaurant.
2,"[The, best, way, to, enjoy, pasta, is, with, a...","[O, O, O, O, O, B-FOOD, O, O, O, O, O]",The best way to enjoy pasta is with a good sauce.
4,"[I, love, to, eat, sushi, at, my, favorite, re...","[O, O, O, O, B-FOOD, O, O, O, O]",I love to eat sushi at my favorite restaurant.
6,"[The, best, Italian, restaurant, in, town, ser...","[O, O, O, O, O, O, O, O, B-FOOD, I-FOOD]",The best Italian restaurant in town serves del...
7,"[I, love, eating, sushi, at, my, favorite, res...","[O, O, O, B-FOOD, O, O, O, O]",I love eating sushi at my favorite restaurant.
...,...,...,...
802,"[Today, I, had, a, delicious, chicken, curry, ...","[O, O, O, O, O, B-FOOD, I-FOOD, O, O, O, O, O,...",Today I had a delicious chicken curry for lunc...
804,"[The, best, Italian, restaurant, in, town, ser...","[O, O, O, O, O, O, O, O, B-FOOD, O, B-FOOD]",The best Italian restaurant in town serves ama...
805,"[The, best, thing, about, summer, is, the, fre...","[O, O, O, O, O, O, O, O, B-FOOD, I-FOOD, O, O,...",The best thing about summer is the fresh fruit...
807,"[I'm thinking of making a, Chicken Parmesan, f...","[O, B-FOOD, I-FOOD]",I'm thinking of making a Chicken Parmesan for ...


In [200]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1EEQRGv6-xOrNCLHOUucg8fvFeZMzCop8ezOdGP9YTyw#gid=0


In [199]:
df.to_csv("clean_data_ner.csv")

# Generate Data With Llama 405 from open router

In [1]:
! pip install openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
openrouter_key=userdata.get("openrouter2")

In [148]:
from openai import OpenAI
from tqdm import tqdm
import json
import time
client=OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_key
)

In [186]:
def gen_data_open_router(num_samples=100):
  data=[]
  j=0
  for i in tqdm(range(num_samples)):
    try:
      response=client.chat.completions.create(
          model="meta-llama/llama-3.1-70b-instruct:free",
          messages=[
            {
                "role":"system",
                "content":content
              },
            {
                "role":"user",
                "content":"generate only 1 record in the dataset"
            },
            {
                "role":"assistant",
                "content":'{"tokens": [ "Sushi", "sarnie", ".", "First", "time", "having", "vegan", "sushi", "yesterday", "since", "are", "in", "Manchester", "for", "a", "short", "time.", "Went", "for", "the", "duck", "sushi", "sandwich", "special", "for", "9.50.", "Might", "need", "to", "go", "back", "and", "get", "it", "again", "before", "they", "change", "the", "special", "on", "Monday.", "Also", "it", "this", "weekend", "so", "it", "was", "a", "good", "enough", "excuse", "to", "have", "it" ],"labels": [ "B-FOOD", "I-FOOD", "O", "O", "O", "O", "B-FOOD", "I-FOOD", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-FOOD", "I-FOOD", "I-FOOD", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ]}'
            },
            {
                "role":"user",
                "content":"generate another one"
            }
          ],
          response_format={"type":"json_object"},
          temperature=0.7,
          top_p=1
      )
      output=json.loads(response.choices[0].message.content)
      data.append(output)
      j+=1
      #time.sleep(1)
    except Exception as e:
      print(e)
      #print(f"error in {i} generated {j}")
      continue
  return data

In [187]:
data_gen=gen_data_open_router(num_samples=200)

  0%|          | 1/200 [00:02<06:47,  2.05s/it]

Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day', 'code': 429}}


  1%|          | 2/200 [00:04<06:35,  2.00s/it]

Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day', 'code': 429}}


  1%|          | 2/200 [00:04<08:12,  2.49s/it]


KeyboardInterrupt: 

In [182]:
len(data_gen)

9

In [286]:
df_or=pd.DataFrame(data_gen)
df_or.head(20)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
df_or.to_csv("new_data_or.csv")

In [ ]:
from google.colab import files
files.download("new_data_or.csv")

In [1]:
df_or[df_or['tokens'].duplicated()]

NameError: name 'df_or' is not defined